In [4]:
# Download Data
!git clone https://github.com/deeplearningzerotoall/PyTorch.git
!mv PyTorch/custom_data/origin_data/ ./

Cloning into 'PyTorch'...
remote: Enumerating objects: 1899, done.
remote: Total 1899 (delta 0), reused 0 (delta 0), pack-reused 1899
Receiving objects: 100% (1899/1899), 80.33 MiB | 27.88 MiB/s, done.
Resolving deltas: 100% (242/242), done.


In [5]:
import torch
from torch import nn

import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader

import random
from matplotlib.pyplot import imshow
import os

# check device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# set seed for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

torch.__version__, device

('1.7.0+cu101', 'cuda')

## ImageFolder
- 원본 사진이 256x512 사이즈의 고해상도 사진이므로, 용량을 낮추기 위해 불러올 때 64x128 사이즈로 변환
- torchvision.transforms.Compose() 를 사용하면 여러 단계의 transform을 묶어 한 번에 적용 가능
<br><br>

## 

In [6]:
trans = transforms.Compose([
    transforms.Resize((64, 128)),
    transforms.ToTensor(),
])

train_data = torchvision.datasets.ImageFolder(root='origin_data/', transform=trans)
data_loader = DataLoader(train_data, batch_size=10, shuffle=True)

train_data[0][0].shape, data_loader

(torch.Size([3, 64, 128]),
 <torch.utils.data.dataloader.DataLoader at 0x7fdfab596150>)

In [7]:
for data in data_loader:
  print(len(data), type(data))
  print(data[0].shape, data[1].shape)
  print(data[1])
  break

2 <class 'list'>
torch.Size([10, 3, 64, 128]) torch.Size([10])
tensor([0, 1, 1, 0, 0, 1, 0, 1, 0, 1])


In [8]:
class CNN(torch.nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.layer1 = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1, padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
    )
    self.layer2 = nn.Sequential(
        nn.Conv2d(6, 16, 5, 1),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),
    )
    self.layer3 = nn.Sequential(
        nn.Linear(in_features=16*13*29, out_features=120, bias=True),
        nn.ReLU(),
        nn.Linear(120, 2)
    )
  
  def forward(self, x, verbose=False):
    def report_shape(output, verbose=verbose):
      if verbose:
        print(output.shape)

    output = self.layer1(x)
    report_shape(output)
    
    output = self.layer2(output)
    report_shape(output)
    
    output = output.view(output.shape[0], -1)
    report_shape(output)
    
    output = self.layer3(output)
    report_shape(output)
    
    return output

In [9]:
# testing - checking output shape of each layer
net = CNN().to(device)
test_in = torch.Tensor(size=(10, 3, 64, 128)).to(device)
test_out = net.forward(test_in, verbose=True)

torch.Size([10, 6, 30, 62])
torch.Size([10, 16, 13, 29])
torch.Size([10, 6032])
torch.Size([10, 2])


In [10]:
%%time

# training
optimizer = torch.optim.Adam(net.parameters(), lr=0.0005)
criterion = nn.CrossEntropyLoss().to(device)

n_epochs = 5
num_batch = len(data_loader)

for epoch in range(n_epochs):
  avg_cost = 0
  for idx, data in enumerate(data_loader):
    value, label = data[0].to(device), data[1].to(device)

    hypothesis = net.forward(value)
    cost = criterion(hypothesis, label)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    avg_cost += cost/num_batch
  print(f'epoch : {epoch+1:3}  |  cost : {avg_cost:10.6f}')
print('>>> Train Finished!')

# save trained weights
os.makedirs('model', exist_ok=True)
torch.save(net, 'model/model.pth')

epoch :   1  |  cost :   0.430994
epoch :   2  |  cost :   0.002398
epoch :   3  |  cost :   0.000228
epoch :   4  |  cost :   0.000036
epoch :   5  |  cost :   0.000010
>>> Train Finished!
CPU times: user 7.96 s, sys: 154 ms, total: 8.11 s
Wall time: 8.18 s
